# Web scraping a Olx (Vehivulos Chevrolet)

In [1]:
# Importacion de las librerias que seran usadas:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import random
import pandas as pd
from vehiculo import Vehiculo
from datetime import datetime

### 1. Instancia del driver:

In [16]:
driver = webdriver.Chrome("./chromedriver.exe")
driver.get("https://www.olx.com.co/")

C:\Users\Adavid\AppData\Local\Temp\ipykernel_14188\594931239.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver.exe")


### 2. Configuracion del input:

In [17]:
ingresar_prod = driver.find_element(by=By.XPATH, value="//input[contains(@placeholder, 'Encuentra automóviles')]")
ingresar_prod.send_keys("chevrolet")

In [18]:
enviar = driver.find_element(by=By.XPATH, value="//div[@data-aut-id='btnSearch']/span")
enviar.click()
time.sleep(5)

### 3. Extraccion de los datos:

In [19]:
# lista de los datos obtenidos:
datos = list()

In [20]:
# Se despliega toda la informacion de la pagina. Esto es, precionando el boton hasta que se despliege todos los productos:
boton = driver.find_element(by=By.XPATH, value="//button[@data-aut-id='btnLoadMore']")
while True:
    try:
        boton.click()
        time.sleep(random.uniform(8.0, 10.0))
        boton = driver.find_element(by=By.XPATH, value="//button[@data-aut-id='btnLoadMore']")
    except:
        break

In [21]:
productos = driver.find_elements(by=By.XPATH, value="//li[@data-aut-id='itemBox']")

In [22]:
for producto in productos:
    try:
        detalle = producto.find_element(by=By.XPATH, value=".//a//div/span[@data-aut-id='itemDetails']").text
        if "km" in detalle:
            titulo = producto.find_element(by=By.XPATH, value=".//a//div/span[@data-aut-id='itemTitle']").text
            precio = producto.find_element(by=By.XPATH, value=".//a//div/span[@data-aut-id='itemPrice']").text
            ubicacion = producto.find_element(by=By.XPATH, value=".//a//div/div/span[1]").text
            obj = Vehiculo(titulo, precio, detalle, ubicacion)
            datos.append(obj.mostrarVehiculo())
    except:
        continue

In [23]:
len(datos)

445

### 4. Limpieza y trasformacion de los datos:

In [28]:
lista_productos = list()
for dicc in datos:
    lista1 = dicc["detalle"].split(" - ")
    lista2 = dicc["ubicacion"].split(", ")
    lista_productos.append({
        "titulo": dicc["titulo"],
        "precio": int(dicc["precio"].replace("$ ", "").replace(".", "")),
        "generacion": int(lista1[0]), 
        "kilometraje": float(lista1[1].replace(" km", "")),
        "ciudad": lista2[1].lower()
    })

### 5. Creacion de dataframe:

In [29]:
olx_df = pd.DataFrame(lista_productos)

In [30]:
olx_df.describe()

,precio,generacion,kilometraje
count,4.450000e+02,445.000000,445.000000
mean,4.100272e+07,2011.357303,117.478667
std,3.342461e+07,8.038224,89.647512
min,1.010101e+06,1956.000000,0.000000
25%,2.350000e+07,2008.000000,62.000000
50%,3.400000e+07,2013.000000,100.000000
75%,4.600000e+07,2017.000000,148.000000
max,4.150000e+08,2022.000000,775.700000


In [31]:
# Exportacion a excel:
olx_df.to_excel(f"olx_chevrolet_{datetime.now().date()}.xlsx", index=False)